In [5]:
!pip install einops tensorflow-cpu transformers accelerate bitsandbytes codecarbon

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.6/516.6 kB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.4/386.4 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.4.0
    Uninstalling termcolor-2.4.0:
      Successfully uninstalled termcolor-2.4.0
  Attempting uninstall: prompt_toolkit
    Found existing installation: prompt_toolkit 3.0.47
    Uninstalling prompt_toolkit-3.0.47:
      Successfully uninstalled prompt_toolkit-3.0.47
ERROR

In [2]:
!git clone https://github.com/muqtasid87/benchmark_vlm

Cloning into 'benchmark_vlm'...
remote: Enumerating objects: 2212, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 2212 (delta 1), reused 11 (delta 1), pack-reused 2192 (from 1)
Receiving objects: 100% (2212/2212), 133.63 MiB | 50.42 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [6]:

from transformers import BitsAndBytesConfig
from transformers import (
    AutoModelForCausalLM,
    AutoProcessor,
    GenerationConfig,
)
from transformers.image_utils import load_image
import torch
import csv
import platform
from transformers import pipeline
import psutil
import numpy as np
from PIL import Image
import os
import shutil
import time
from codecarbon import EmissionsTracker
import requests
from transformers import AutoModelForCausalLM
from transformers import AutoProcessor

In [7]:
model_id = ["cyan2k/molmo-7B-D-bnb-4bit"]

In [8]:
# Image directory
img_dir = 'benchmark_vlm/master'

### Inference Function

In [9]:
# model = AutoModelForCausalLM.from_pretrained(
#   model_id[0],
#   device_map="cuda",
#   trust_remote_code=True,
#   torch_dtype="auto",
#   _attn_implementation='eager'
# )

# # for best performance, use num_crops=4 for multi-frame, num_crops=16 for single-frame.
# processor = AutoProcessor.from_pretrained(model_id[0],
#   trust_remote_code=True,
#   num_crops=16
# )

In [10]:
# Function to run inference
def infer_image(image_path, prompt):
    with open(image_path, "rb") as f:
          input_image = Image.open(f).convert("RGB")
          
    
    inputs = processor.process(
    images=input_image,
    text=prompt,
)
    inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}
    
    # generate output; maximum 200 new tokens; stop generation when <|endoftext|> is generated
    output = model.generate_from_batch(
    inputs,
    GenerationConfig(max_new_tokens=2, stop_strings="<|endoftext|>"),
    tokenizer=processor.tokenizer,
)

# only get generated tokens; decode them to text
    generated_tokens = output[0, inputs["input_ids"].size(1) :]
    generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return generated_text.strip().lower()

### Validation function

In [11]:
# Define vehicle classes for validation
vehicle_classes = {
    "Class 1 - Motorcycles": ["motorcycle", "motorbike", "bike", "scooter", "motor", "moped"],
    "Class 2 - Car, SUV, Station Wagons": ["car", "sedan", "hatchback", "suv", "mpv", "crossover", "station wagon",  'coupe', 'convertible', 'sports car', "stationwagon",'sportscar'],
    "Class 3 - 2 axle lorry, van, pickup": ["pickup", "van", "minivan"],
    "Class 4 - 2 axles, 5 or 6 wheels bus or truck": ["bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "Class 5 - 3 axle truck, bus": ["bus", "truck"],
    "Class 6 - 4 axles": ["coach", "bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "Class 7 - 5 or more axles": ["coach", "bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer", "towtruck","dumptrailer"],
    "bus 2" : ["coach", "bus", "minibus"],
    "bus 3" : ["coach", "bus"],
    "truck 2" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer", "towtruck","dumptrailer"],
    "truck 3" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer", "towtruck","dumptrailer"],
    "truck 4" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer", "towtruck","dumptrailer"],
    "truck 5" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer", "towtruck","dumptrailer"],
    "van" : ["van", "minivan", 'minibus'],
    "pickup" : ["pickup", "pickup truck", "camper", "pickuptruck"],
}

valid_vehicle_types = {type for sublist in vehicle_classes.values() for type in sublist}

In [12]:
# Validate predictions

def validate_prediction(folder_name, prediction, axles=None):
    # Extract folder label
    for class_name, valid_labels in vehicle_classes.items():
        if folder_name.startswith(class_name):
            if axles:  # Use axle count for bus/truck
                if axles == 1 or 0:
                  return False
                elif axles == 2:
                    return "2" in folder_name[-1]
                elif axles == 3:
                    return "3" in folder_name[-1]
                elif axles == 4:
                    return "4" in folder_name[-1]
                elif axles >= 5:
                    return "5" in folder_name[-1]
            if isinstance(valid_labels, list):
                return prediction in valid_labels
            return prediction == valid_labels
    return False

# Main Loop

In [13]:
from tqdm import tqdm

In [14]:
def main_loop(wrong_predictions_dir, img_dir=img_dir, use_axles=True, valid_vehicle_types=valid_vehicle_types):
    total_files = sum(len(files) for _, _, files in os.walk(img_dir))
    progress_bar = tqdm(total=total_files, desc="Processing Images", unit="file")
    # Main processing loop
    start_time = time.time()
    tracker = EmissionsTracker()
    tracker.start()
    results = []
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if file.endswith((".jpg", ".png", ".jpeg")):
                image_path = os.path.join(root, file)
                folder_name = os.path.basename(os.path.dirname(image_path))
                progress_bar.set_description(f"Processing: {folder_name}")
                progress_bar.update(1)  # Increment the progress bar

                try:
                    # First inference: vehicle type
                    # vehicle_prompt = f"<image> answer en What type of vehicle is this? Answer in only one word."
                    vehicle_prompt = f"What type of vehicle is this? Choose one from: {', '.join(valid_vehicle_types)}. Answer in one word"
                    prediction = infer_image(image_path, vehicle_prompt)
                    print(f"Vehicle Prediction: {prediction}")

                    # Handle buses and trucks
                    axles = None
                    if prediction in ["bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck',  "tow truck", "dump trailer"]:
                        # Check if it's a pickup truck
                        if prediction == 'truck':
                            pickup_prompt = "Is this a pickup truck? Respond with only yes or no."
                            is_pickup = infer_image(image_path, pickup_prompt).strip().lower()
                            print(f"Is Pickup Truck: {is_pickup}")
                            if is_pickup == "yes":
                                prediction = "pickup truck"
                            elif use_axles:
                                # Predict axles for non-pickup trucks
                                axle_prompt = "How many axles does this vehicle have? Output a single integer value."
                                axle_prediction = infer_image(image_path, axle_prompt)
                                print(f"Axle Prediction: {axle_prediction}")
                    
                                try:
                                    axles = int(axle_prediction)
                                except ValueError:
                                    print(f"Invalid axle prediction: {axle_prediction}")
                                    axles = None  # Fallback if parsing fails
                        elif use_axles:

                            axle_prompt = "How many axles does this vehicle have? Output a single integer value"
                            axle_prediction = infer_image(image_path, axle_prompt)
                            print(f"Axle Prediction: {axle_prediction}")
    
                            try:
                                axles = int(axle_prediction)
                            except ValueError:
                                print(f"Invalid axle prediction: {axle_prediction}")

                    # Validate prediction
                    is_correct = validate_prediction(folder_name, prediction, axles)
                    print(f"Validation Result: {is_correct}")

                    # Store results
                    results.append({
                        "image": image_path,
                        "folder_label": folder_name,
                        "prediction": prediction,
                        "axles": axles,
                        "is_correct": is_correct
                    })

                    # Handle wrong predictions
                    if not is_correct:
                        # Create a folder for the wrong prediction
                        wrong_folder = os.path.join(wrong_predictions_dir, prediction, str(axles))
                        os.makedirs(wrong_folder, exist_ok=True)

                        # Copy the image to the folder
                        shutil.copy(image_path, os.path.join(wrong_folder, file))
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")
                    continue
    
    progress_bar.close()
    emissions = tracker.stop()
    # Stop the timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Total processing time: {elapsed_time:.2f} seconds")
    print(f"CO2 emissions: {emissions:.6f} kg")
    return (results, elapsed_time, emissions)

In [15]:
# Save results to CSV
def export_results(results, model_id, elapsed_time, emissions):
# Save results to CSV
    output_csv = f'experiment_data/{models}/axles/results.csv'
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["image", "folder_label", "prediction", "axles", "is_correct"])
        writer.writeheader()
        writer.writerows(results)

    # Metrics calculation
    correct_predictions = sum(1 for r in results if r["is_correct"])
    total_images = len(results)
    accuracy = correct_predictions / total_images * 100 if total_images > 0 else 0
    wrong_predictions = total_images - correct_predictions

    # Per-class metrics
    class_metrics = {}
    for result in results:
        label = result["folder_label"]
        correct = result["is_correct"]
        if label not in class_metrics:
            class_metrics[label] = {"correct": 0, "total": 0}
        class_metrics[label]["total"] += 1
        if correct:
            class_metrics[label]["correct"] += 1

    # Aggregate class metrics
    per_class_accuracy = {
        label: (data["correct"] / data["total"]) * 100
        for label, data in class_metrics.items()
    }
    per_class_error_rate = {
        label: 100 - acc for label, acc in per_class_accuracy.items()
    }

    # Collect system configuration
    system_config = {
        "Processor": platform.processor(),
        "CPU Count": psutil.cpu_count(logical=True),
        "Memory (GB)": round(psutil.virtual_memory().total / (1024 ** 3), 2),
        "Elapsed Time (seconds)": elapsed_time,
        "CO2 Emissions (kg)": round(emissions, 6)
    }

    # Metrics dictionarya
    metrics = {
        "Total Images": total_images,
        "Correct Predictions": correct_predictions,
        "Wrong Predictions": wrong_predictions,
        "Accuracy (%)": accuracy,
        "Error Rate (%)": 100 - accuracy
    }

    # Save metrics to CSV
    metrics_csv = f'experiment_data/{models}/axles/metrics.csv'
    with open(metrics_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Model", "Metric", "Value"])
        for key, value in metrics.items():
            writer.writerow([model_id, key, value])
        for label, acc in per_class_accuracy.items():
            writer.writerow([model_id, f"Per-Class Accuracy: {label}", acc])
            writer.writerow([model_id, f"Per-Class Error Rate: {label}", per_class_error_rate[label]])
        for key, value in system_config.items():
            writer.writerow([model_id, key, value])

    # Print final metrics
    print("\n--- Metrics ---")
    for key, value in metrics.items():
        print(f"{key}: {value}")
    for label, acc in per_class_accuracy.items():
        print(f"Class '{label}' - Accuracy: {acc:.2f}%, Error Rate: {per_class_error_rate[label]:.2f}%")
    for key, value in system_config.items():
        print(f"{key}: {value}")

In [16]:
for models in model_id:
    print(models)
    arguments = {"device_map": "balanced", "torch_dtype": "auto", "trust_remote_code": True}
    processor = AutoProcessor.from_pretrained(models, **arguments)
    model = AutoModelForCausalLM.from_pretrained(models, **arguments)
        
    
    wrong_predictions_dir = f'experiment_data/{models}/axles/wrong_predictions'  # Directory to store wrongly predicted images
    os.makedirs(wrong_predictions_dir, exist_ok=True)
    run = main_loop(wrong_predictions_dir = wrong_predictions_dir)
    export_results(run[0], models, run[1], run[2])
    
        

cyan2k/molmo-7B-D-bnb-4bit


processor_config.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

preprocessing_molmo.py:   0%|          | 0.00/5.79k [00:00<?, ?B/s]

image_preprocessing_molmo.py:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/cyan2k/molmo-7B-D-bnb-4bit:
- image_preprocessing_molmo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/cyan2k/molmo-7B-D-bnb-4bit:
- preprocessing_molmo.py
- image_preprocessing_molmo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Processing: truck 5:  99%|█████████▉| 927/932 [1:33:45<00:28,  5.61s/file]

Vehicle Prediction: trailer
Validation Result: True


Processing: truck 5:  99%|█████████▉| 927/932 [1:33:48<00:30,  6.07s/file]
[codecarbon INFO @ 16:46:45] Energy consumed for RAM : 0.018354 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 16:46:45] Energy consumed for all CPUs : 0.066359 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 16:46:45] Energy consumed for all GPUs : 0.149208 kWh. Total GPU Power : 95.74779024174299 W
[codecarbon INFO @ 16:46:45] 0.233922 kWh of electricity used since the beginning.


Vehicle Prediction: dumptruck
Validation Result: False
Total processing time: 5628.27 seconds
CO2 emissions: 0.066785 kg

--- Metrics ---
Total Images: 927
Correct Predictions: 742
Wrong Predictions: 185
Accuracy (%): 80.04314994606257
Error Rate (%): 19.95685005393743
Class 'van' - Accuracy: 95.00%, Error Rate: 5.00%
Class 'pickup' - Accuracy: 93.00%, Error Rate: 7.00%
Class 'truck 2' - Accuracy: 72.28%, Error Rate: 27.72%
Class 'bus 2' - Accuracy: 83.33%, Error Rate: 16.67%
Class 'Class 2 - Car, SUV, Station Wagons' - Accuracy: 94.95%, Error Rate: 5.05%
Class 'truck 4' - Accuracy: 68.97%, Error Rate: 31.03%
Class 'Class 1 - Motorcycles' - Accuracy: 99.00%, Error Rate: 1.00%
Class 'truck 3' - Accuracy: 42.00%, Error Rate: 58.00%
Class 'bus 3' - Accuracy: 97.96%, Error Rate: 2.04%
Class 'truck 5' - Accuracy: 55.00%, Error Rate: 45.00%
Processor: x86_64
CPU Count: 4
Memory (GB): 31.35
Elapsed Time (seconds): 5628.269790172577
CO2 Emissions (kg): 0.066785


/usr/local/lib/python3.10/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])
